In [49]:
import pandas as pd
import os
# from analyzer.utils import 
from analyzer.decoder import trace_transaction, mutate_call_trace, make_mutate_map
from web3._utils.contracts import encode_abi
from analyzer.utils import w3, parse_traces, get_code_from_adr
from analyzer.detection_rules import analyze_mutate_function_input, find_getReserves, mutate

In [3]:
df = pd.read_csv('./formated_tx.csv')

In [3]:
tx = w3.eth.getTransaction('0x957cee22eac247f9905a5e2a41a42e661001d6baed5eea005642668fc975ddcd')

In [26]:
def get_code_input(tx_hash):
#     print(tx_hash)
    tx = w3.eth.getTransaction(tx_hash)
    c = get_code_from_adr(tx['to'])
    return c.decode_input(tx['input'])
f= get_code_input('0x957cee22eac247f9905a5e2a41a42e661001d6baed5eea005642668fc975ddcd')

In [53]:
parsed

{'from': '0x9213dc91b318f030eeac9085871939c6c7c58089',
 'from_contract': '',
 'to': '0x0f41d2a6eca64a5e563bfa16d86ca82cdcf97528',
 'to_name': 'DSProxy',
 'calls': [{'from': '0x0f41d2a6eca64a5e563bfa16d86ca82cdcf97528',
   'from_contract': 'DSProxy',
   'to': '0x894327a2a57b700c130e00efae0b95a16e7b5c11',
   'to_name': 'MCDCloseTaker',
   'calls': [{'from': '0x0f41d2a6eca64a5e563bfa16d86ca82cdcf97528',
     'from_contract': 'DSProxy',
     'to': '0xe9e448245fdc8b629ce0888d25861740773cc17c',
     'to_name': 'MCDCloseFlashLoan',
     'calls': [],
     'error': '',
     'input': '0x',
     'raw_input': '0x',
     'output': '0x'},
    {'from': '0x0f41d2a6eca64a5e563bfa16d86ca82cdcf97528',
     'from_contract': 'DSProxy',
     'to': '0x5ef30b9986345249bc32d8928b7ee64de9435e39',
     'to_name': 'DssCdpManager',
     'calls': [],
     'error': '',
     'input': (<Function urns(uint256)>, {'': 17566}),
     'raw_input': '0x2726b073000000000000000000000000000000000000000000000000000000000000449e'

In [51]:
traces = trace_transaction('0x84cc49467123c4e7f9a1a92d6ce59dfeab6cb9fa5bd3cf048601dc735b4fa0df')
parsed = parse_traces(traces['result'])

In [73]:
get_code_from_adr(parsed['to'])._contract.encodeABI

<function web3.contract.Contract.encodeABI(cls, fn_name: str, args: Union[Any, NoneType] = None, kwargs: Union[Any, NoneType] = None, data: Union[HexStr, NoneType] = None) -> <function NewType.<locals>.new_type at 0x7f66885960d0>>

In [83]:
encode_abi(w3,fn[0].abi, fn[1])

EncodingTypeError: Value `'tier'` of type <class 'str'> cannot be encoded by UnsignedIntegerEncoder

In [8]:
def get_suspicious_fields(fn_tuple):
    threshold = 1000
    abi = fn_tuple[0].abi
    suspicious_field = []
    for input_field in abi['inputs']:
        if input_field['internalType'] == 'uint256':
            if fn_tuple[1][input_field['name']] > threshold:
                suspicious_field.append(input_field['name'])
    return suspicious_field

def detection_mutate_input(tx, parsed):
    amplified_rate = 1.5
    suspicious_fields = get_suspicious_fields(parsed['input'])
    if len(suspicious_fields) == 0:
        return
    
    contract = get_code_from_adr(tx['to'])._contract
    original_data = parsed['input']
    for field in suspicious_fields:
        tmp = original_data[1]
        tmp[field] = int(tmp[field]*amplified_rate)
        input = contract.encodeABI(original_data[0].fn_name, tmp)

In [33]:
def mutate_input_by_fn_input(adr, fn_dict):
    print(fn_dict[adr])

mutate_input_by_fn_input(distinct_address[0], address_fn)

[(<Function stake(uint256,uint256,uint256,uint256,uint256,uint256)>, {'tier': 0, 'poolPosition': 0, 'originalFirstAmount': 1000000000000000000, 'firstAmountMin': 995000000000000000, 'value': 0, 'secondAmountMin': 11043293857456946}), (<Function stake(uint256,uint256,uint256,uint256,uint256,uint256)>, {'tier': 0, 'poolPosition': 0, 'originalFirstAmount': 1000000000000000000, 'firstAmountMin': 995000000000000000, 'value': 0, 'secondAmountMin': 11043293857456946})]


In [65]:
get_code_from_adr(distinct_address[0])._contract.encodeABI('stake', f[1][1])

'0xfdd56605000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000de0b6b3a76400000000000000000000000000000000000000000000000000000dcef33a6f838000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000273bd137862332'

In [69]:
def get_uint_by_fn(f):
    threshold = 1000
    abi = f[0].abi
    suspicious_field = []
    for input_field in abi['inputs']:
        if input_field['internalType'] == 'uint256':
            if f[1][input_field['name']] > threshold:
                suspicious_field.append(input_field['name'])
    return suspicious_field
get_uint_by_fn(f[0])

['originalFirstAmount', 'firstAmountMin', 'secondAmountMin']

In [34]:
f = address_fn[distinct_address[0]]

In [88]:
fn[1]

{'tier': 0,
 'poolPosition': 0,
 'originalFirstAmount': 1000000000000000000,
 'firstAmountMin': 995000000000000000,
 'value': 0,
 'secondAmountMin': 11043293857456946}

In [63]:
f[1][1]

{'tier': 0,
 'poolPosition': 0,
 'originalFirstAmount': 1000000000000000000,
 'firstAmountMin': 995000000000000000,
 'value': 0,
 'secondAmountMin': 11043293857456946}

In [28]:
count = 0
distinct_address = []
address_fn = {}
for entrypoint, g in groups:
    count += 1 
#     f = []
    print('entrypoint: ', entrypoint)

    for tx in g['tx_hash'].values[:3]:
        try:
            f = get_code_input(tx)
            if type(f) != type('s'):
                if entrypoint not in distinct_address:
                    distinct_address.append(entrypoint)
                    address_fn[entrypoint] = [f]
                else:
                    address_fn[entrypoint].append(f)
        except:
            pass
    if count > 4197:
        break

SyntaxError: unmatched ')' (<ipython-input-28-04d6ab36da24>, line 1)

## fuzz with input

In [7]:
def get_suspicious_fields(fn_tuple):
    threshold = 1000
    abi = fn_tuple[0].abi
    suspicious_field = []
    for input_field in abi['inputs']:
        if input_field['type'] == 'uint256' or input_field['type'] == 'uint128':
            if fn_tuple[1][input_field['name']] > threshold:
                suspicious_field.append(input_field['name'])
    return suspicious_field

In [8]:
def detection_mutate_input(tx, parsed):
    if parsed['error'] != '':
        return []
    amplified_rate = 1.5
    suspicious_fields = get_suspicious_fields(parsed['input'])
    if len(suspicious_fields) == 0:
        return []
    
    contract = get_code_from_adr(parsed['to'])._contract
    original_data = parsed['input']
    inputs = []
    for field in suspicious_fields:
        tmp = original_data[1]
        tmp[field] = int(tmp[field]*amplified_rate)
        input = contract.encodeABI(original_data[0].fn_name, tmp)
        
        traces = mutate_call_trace(tx, make_mutate_map([], []), input)
        if 'error' in traces['result'].keys():
            inputs.append((input, field))

    
    return inputs
detection_mutate_input('0xc869de9b05b82e370071b4203ef3366dc8a7cdbcb699795af622d5c9d6089072', parsed)

[('0x25e07c3e0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a5da8cc7167070b62fdcb332ef097a55a68d88240000000000000000000000000000000000000000000000006263c71e7f2ed00000000000000000000000000000000000000000000000000010a8527d93e6c000',
  '_IncomingBPT'),
 ('0x25e07c3e0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a5da8cc7167070b62fdcb332ef097a55a68d88240000000000000000000000000000000000000000000000006263c71e7f2ed00000000000000000000000000000000000000000000000000018fc7bbc5dda2000',
  '_minTokensRec')]

In [16]:
tx = '0x88ba7b5f62a2a41399b07e9ff6e998f2559dee6798d10d1011b86834e8cd5b4e'
traces = trace_transaction(tx)
parsed = parse_traces(traces['result'])

In [23]:
parsed['input'][1]

{'_amount': 260532200783961439703034716269547792669857465497691269088779564017804541689856}

In [27]:
tmp = parsed['input']

In [31]:
tmp[1]

{'_amount': 260532200783961439703034716269547792669857465497691269088779564017804541689856}

In [42]:
type(115792089237316195423570985008687907853269984665640564039457584007913129639935)

int

In [48]:
traces = trace_transaction('0x8df2e5917509fd9a8403d23fb7ca9ba9f4f60b3f7af110b00e46d986e866fd7f')
parse_traces(traces['result'])

{'from': '0x7180e344f5596171b1a0f0bffd42e389300a4822',
 'from_contract': '',
 'to': '0xc6c11f32d3ccc3beaac68793bc3bfbe82838ca9f',
 'to_name': 'CBOND',
 'calls': [{'from': '0xc6c11f32d3ccc3beaac68793bc3bfbe82838ca9f',
   'from_contract': 'CBOND',
   'to': '0xcf9eaa3f2e5bfa56f981fd1cea11c65caa70aafb',
   'to_name': '',
   'calls': [{'from': '0xcf9eaa3f2e5bfa56f981fd1cea11c65caa70aafb',
     'from_contract': '',
     'to': '0xfb2f545a9ad62f38fe600e24f75ecd790d30a7ba',
     'to_name': 'UniswapV2Pair',
     'calls': [],
     'error': '',
     'input': (<Function getReserves()>, {}),
     'raw_input': '0x0902f1ac',
     'output': '0x0000000000000000000000000000000000000000000629957a0f942a1ffdb094000000000000000000000000000000000000000000000006f50852999de9b87f000000000000000000000000000000000000000000000000000000005fcd045d'},
    {'from': '0xcf9eaa3f2e5bfa56f981fd1cea11c65caa70aafb',
     'from_contract': '',
     'to': '0xfb2f545a9ad62f38fe600e24f75ecd790d30a7ba',
     'to_name': 'UniswapV2P

In [37]:
c = get_code_from_adr(parsed['to'])._contract
c.encodeABI(tmp[0].fn_name, [260532200783961439703034716269547792669857465497691269088779564017804541689856])

ValidationError: 
Could not identify the intended function with name `redeem`, positional argument(s) of type `(<class 'int'>,)` and keyword argument(s) of type `{}`.
Found 1 function(s) with the name `redeem`: ['redeem(uint256)']
Function invocation failed due to no matching argument types.

In [25]:
parsed['input']

(<Function redeem(uint256)>,
 {'_amount': 260532200783961439703034716269547792669857465497691269088779564017804541689856})

In [12]:
find_getReserves

[('0x25e07c3e0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a5da8cc7167070b62fdcb332ef097a55a68d88240000000000000000000000000000000000000000000000006263c71e7f2ed00000000000000000000000000000000000000000000000000010a8527d93e6c000',
  '_IncomingBPT'),
 ('0x25e07c3e0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a5da8cc7167070b62fdcb332ef097a55a68d88240000000000000000000000000000000000000000000000006263c71e7f2ed00000000000000000000000000000000000000000000000000018fc7bbc5dda2000',
  '_minTokensRec')]

In [18]:
calls = find_getReserves(parsed)

mutated_inputs = detection_mutate_input(tx, parsed)


ValidationError: 
Could not identify the intended function with name `redeem`, positional argument(s) of type `(<class 'str'>,)` and keyword argument(s) of type `{}`.
Found 1 function(s) with the name `redeem`: ['redeem(uint256)']
Function invocation failed due to no matching argument types.

In [15]:
parsed

{'from': '0x165edc0257b97d155978da40007b608169170a66',
 'from_contract': '',
 'to': '0x00d0f137b51692d0ac708bde7b367a373865cffe',
 'to_name': 'Balancer_ZapOut_General_V2_2',
 'calls': [{'from': '0x00d0f137b51692d0ac708bde7b367a373865cffe',
   'from_contract': 'Balancer_ZapOut_General_V2_2',
   'to': '0x9424b1412450d0f8fc2255faf6046b98213b76bd',
   'to_name': 'BFactory',
   'calls': [],
   'error': '',
   'input': (<Function isBPool(address)>,
    {'b': '0xa5DA8Cc7167070B62FdCB332EF097A55A68d8824'}),
   'raw_input': '0xc2bb6dc2000000000000000000000000a5da8cc7167070b62fdcb332ef097a55a68d8824',
   'output': '0x0000000000000000000000000000000000000000000000000000000000000001'},
  {'from': '0x00d0f137b51692d0ac708bde7b367a373865cffe',
   'from_contract': 'Balancer_ZapOut_General_V2_2',
   'to': '0xa5da8cc7167070b62fdcb332ef097a55a68d8824',
   'to_name': 'BPool',
   'calls': [],
   'error': '',
   'input': (<Function isBound(address)>,
    {'t': '0x0000000000000000000000000000000000000000'})

In [19]:
from analyzer.detection_rules import collect_event
transfers = collect_event(traces['result'], 'transfer')

for input in mutated_inputs:
    print('mutated suspicious field', input[1])
    suspicious = mutate(tx, calls, 
        [1.01, 0.99, 1.05, 0.95, 2, 0.5, 5, 0.2, 10, 0.1, 1000, 0.001],
        transfers, 'callee', input[0]
        )

mutated suspicious field _amount
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 1.01
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 0.99
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 1.05
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 0.95
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 2
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 0.5
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 5
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 0.2
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 10
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 0.1
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 1000
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 0.001
mutating getReserves : 0xae461ca67b15dc8dc81ce7615e03

In [18]:
calls

[{'input': (<Function getReserves()>, {}),
  'from': '0x4a224cd0517f08b26608a2f73bf390b01a6618c8',
  'to': '0xa478c2975ab1ea89e8196811f51a7b7ade33eb11'},
 {'input': (<Function getReserves()>, {}),
  'from': '0x496b5607e6ef186d5de849a2791fb186e2e94982',
  'to': '0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5'},
 {'input': (<Function getReserves()>, {}),
  'from': '0x4a224cd0517f08b26608a2f73bf390b01a6618c8',
  'to': '0xbb2b8038a1640196fbe3e38816f3e67cba72d940'},
 {'input': (<Function getReserves()>, {}),
  'from': '0x2261a20c1aa9a73bc35bdb36cd5830d94f2f7ddb',
  'to': '0x004375dff511095cc5a197a54140a24efef3a416'},
 {'input': (<Function getReserves()>, {}),
  'from': '0x4a224cd0517f08b26608a2f73bf390b01a6618c8',
  'to': '0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852'},
 {'input': (<Function getReserves()>, {}),
  'from': '0x4a224cd0517f08b26608a2f73bf390b01a6618c8',
  'to': '0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc'},
 {'input': (<Function getReserves()>, {}),
  'from': '0x496b5607e6ef186d5de8

In [3]:
def log(l):
    print(l)

In [4]:
analyze_mutate_function_input('0xc869de9b05b82e370071b4203ef3366dc8a7cdbcb699795af622d5c9d6089072', log)

 start anaylzing tx: 0xc869de9b05b82e370071b4203ef3366dc8a7cdbcb699795af622d5c9d6089072
entry point: 0xba539b9a5c2d412cb10e5770435f362094f9541c
collects 1 transfers events
decoding traces
mutated suspicious field _amount
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 1.01
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 0.99
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 1.05
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 0.95
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 2
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 0.5
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 5
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 0.2
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 10
mutating getReserves : 0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 rate: 0.1
mutating getReserves

In [7]:
tx = '0xc869de9b05b82e370071b4203ef3366dc8a7cdbcb699795af622d5c9d6089072'
suspicious_fileds =detection_mutate_input(tx, parsed)

NameError: name 'detection_mutate_input' is not defined

In [39]:
for c in distinct_calls:
    for rate in [1.1, 0.9, 2, 0.5, 5, 0.2]:
        traces = mutate_call_trace(tx, make_mutate_map([c], [rate]), suspicious_fileds[0][0])
        print(c, rate, traces['result']['error'])

0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 1.1 execution reverted
0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 0.9 execution reverted
0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 2 execution reverted
0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 0.5 execution reverted
0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 5 execution reverted
0xa478c2975ab1ea89e8196811f51a7b7ade33eb11 0.2 execution reverted
0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5 1.1 execution reverted
0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5 0.9 execution reverted
0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5 2 execution reverted
0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5 0.5 execution reverted
0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5 5 execution reverted
0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5 0.2 execution reverted
0xbb2b8038a1640196fbe3e38816f3e67cba72d940 1.1 execution reverted
0xbb2b8038a1640196fbe3e38816f3e67cba72d940 0.9 execution reverted
0xbb2b8038a1640196fbe3e38816f3e67cba72d940 2 execution reverted
0xbb2b8038a1640196fb

KeyError: 'error'

In [36]:
distinct_calls

['0xa478c2975ab1ea89e8196811f51a7b7ade33eb11',
 '0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5',
 '0xbb2b8038a1640196fbe3e38816f3e67cba72d940',
 '0x004375dff511095cc5a197a54140a24efef3a416',
 '0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852',
 '0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc']

In [35]:
distinct_calls = [] 
for c in calls:
    if c['to'] not in distinct_calls:
        distinct_calls.append(c['to'])

In [9]:
calls = find_getReserves(parsed)

In [12]:
traces

{'jsonrpc': '2.0',
 'id': 46,
 'result': {'type': 'CALL',
  'from': '0xb30f61798db19c404cf5723a24b92fa9c7c1e26d',
  'to': '0xba539b9a5c2d412cb10e5770435f362094f9541c',
  'value': '0x0',
  'gas': '0x7ac24',
  'gasUsed': '0x7a453',
  'input': '0x0a5559890000000000000000000000006b175474e89094c44da98b954eedeac495271d0f000000000000000000000000000000000000000000000043c33c193756480000',
  'output': '0x',
  'time': '594.07349ms',
  'calls': [{'type': 'CALL',
    'from': '0xba539b9a5c2d412cb10e5770435f362094f9541c',
    'to': '0x6046c3ab74e6ce761d218b9117d5c63200f4b406',
    'value': '0x0',
    'gas': '0x77c7e',
    'gasUsed': '0xaafd',
    'input': '0x17bfdfbc000000000000000000000000b30f61798db19c404cf5723a24b92fa9c7c1e26d',
    'output': '0x0000000000000000000000000000000000000000000000000000000000000000',
    'calls': [{'type': 'STATICCALL',
      'from': '0x6046c3ab74e6ce761d218b9117d5c63200f4b406',
      'to': '0x6b175474e89094c44da98b954eedeac495271d0f',
      'gas': '0x750e1',
      'gas

In [10]:
calls

[{'input': (<Function getReserves()>, {}),
  'from': '0x4a224cd0517f08b26608a2f73bf390b01a6618c8',
  'to': '0xa478c2975ab1ea89e8196811f51a7b7ade33eb11'},
 {'input': (<Function getReserves()>, {}),
  'from': '0x496b5607e6ef186d5de849a2791fb186e2e94982',
  'to': '0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5'},
 {'input': (<Function getReserves()>, {}),
  'from': '0x4a224cd0517f08b26608a2f73bf390b01a6618c8',
  'to': '0xbb2b8038a1640196fbe3e38816f3e67cba72d940'},
 {'input': (<Function getReserves()>, {}),
  'from': '0x2261a20c1aa9a73bc35bdb36cd5830d94f2f7ddb',
  'to': '0x004375dff511095cc5a197a54140a24efef3a416'},
 {'input': (<Function getReserves()>, {}),
  'from': '0x4a224cd0517f08b26608a2f73bf390b01a6618c8',
  'to': '0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852'},
 {'input': (<Function getReserves()>, {}),
  'from': '0x4a224cd0517f08b26608a2f73bf390b01a6618c8',
  'to': '0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc'},
 {'input': (<Function getReserves()>, {}),
  'from': '0x496b5607e6ef186d5de8